# Advanced Deep Learning Best Practices

## 7.1.1 Introduction to the functional API

In [ ]:
from keras import Input, layers
input_tensor = Input (shape=(32,))     # A tensor
dense = layers.Dense(32, activation = 'relu')   # A layer is a function
output_tensor = dense(input_tensor)         # A layer may be called a tensor and it returns a tensor

### Minimal Eg., Sequential Model with its eq in functional API side by side:-

In [ ]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()       # Seq Md
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)    # Func eq

model = Model(input_tensor, output_tensor)   # The Model class turns an input tensor and output tensor in2 a model

model.summary()

#### The APi is same as that of sequntial when it comes to compiling training or evaluating such an instance of the model

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')  # Compiles the model

import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))         # Gen dummy numpy data to train on

model.fit(x_train, y_train,epochs=10, batch_size=128)     # Trains the model for 10 epochs
score = model.evaluate(x_train, y_train)       # evaluates the model

### Multi-input models

#### Functional API helps build models having multiple inputs. Use Keras merge operation such as keras.layers.add, keras.layers.concatenate.

## 7.1 Functional API implwemetation of a two-point question-answering model

In [ ]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype = 'int32', name = 'text')     #The text input is a variable-length sequence
                                                                     #of integers. Note: U can optionally name the inputs.
embedded_text = layers.Embedding(         # Embeds the inputs into a sequence of vectors of size 64.                
    64, text_vocabulary_size)(text_input)    

encoded_text = layers.LSTM(32)(embedded_text)  #Encodes the vectors in a single vector via an LSTM.

question_input = Input(shape=(None,),     # Same process (with different layer instances) for the question.
                              dtype='int32',
                              name='question')

embedded_question = layers.Embedding(32,  question_vocabulary_size)(question_input)

encoded_question= layers.LSTM(16)(embedded_question)   

concatenated = layers.concatenate([encoded_text, encoded_question], axis = -1) #Concatenation

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated) # Adds a softmax classifier on top.

model = Model([text_input, question_input], answer) # Specify the two inputs and outputs at model instantiation.

model.compile(optimizer='rmsprop',
                         loss='categorical_crossentropy',
                         metrics=['acc'])

## Listing 7.2  Feeding Data to a multi-input model

In [ ]:
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,
                         size=(num_samples, max_length))      # Generates dummy Numpy data

question = np.random.randint(1, question_vocabulary_size,
                         size=(num_samples, max_length))

answers = np.random.randint(0, 1, 
                         size=(num_samples, answer_vocabulary_size))  # Answers are 1hot encoded, not integers.

model.fit([text, question], answers, epochs=10, batch_size=128)  # Fitting using a list of inputs.

model.fit({'text': text, 'question': question}, answers,
          epochs=10, batch_size=128)    # Fitting using a dictionary of inputs(only if inputs are named)

### 7.1.3 Multi-output models: Functional API also builds models with multiple(outputs)(heads). e.g., a network attempting
                           to simultaneously predict different props of the data such as social media posts predicting 
                           attributes of a person such as age, gender and income level.                 

## Listing 7.3: Functional API implementation of a three output model.

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])


## Listing 7.4: Compilation options of a multi-output model: multiple losses

In [ ]:
model.compile(optimizer='rmsprop',
                 loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'}) # Equivalent possible only if you give names to the output layers


## Listing 7.5: Compilation options of a multi-output model: loss weighting

In [ ]:
model.compile(optimizer='rmsprop',
                 loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'}, # Equivalent possible only if you give names to the output layers
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})      

### Listing 7.6: Feeding data to a multi-output model

In [ ]:
model.fit(posts, [age_targerts, income_targets, gender_targets],
          epochs=10, batch_size=64)
#age_targets, income_targets and gender_targets are assumed to be numpy arrays
modelfit(posts, { 'age': age_targets,
                  'income': income_targets,
                  'gender': gender_targets},
         epochs=10, batch_size=64)   # Equivalent(possible only if you give name to the output layers)